# _EXPERT_ usage reference: Advanced stepwise invocation

## Log in to the _EXPERT_-client
_EXPERT_ needs credentials for authentication. In order to log in, you create an authenticated client which handles all interactions with the _future_ platform.

If you are using _EXPERT_ for the first time, we recommend that you begin with [Getting Started](getting_started.ipynb).

In [ ]:
from futureexpert import (DataDefinition,
                          ExpertClient,
                          FileSpecification,
                          ForecastingConfig,
                          ReportConfig,
                          TsCreationConfig)

client = ExpertClient(user='', password='')

## Run each step manually

Calling `client.start_forecast_from_raw_data` invokes the whole pipeline comprising several steps.

Alternatively, you can run each step individually in order to get intermediate results and fast feedback.

### Select your raw data

There are two possible options:
- upload new data to future, or
- use data that has already been uploaded.

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)  # Some basic formatting configuration for notebook/console output.

In [ ]:
# upload new data to future
upload_feedback = client.upload_data(source='../example_data/example_data.csv')
pp.pprint(upload_feedback)

In [ ]:
# receive all files you have already uploaded
input_data = client.api_client.get_user_inputs_for_group(client.group)
pp.pprint(input_data)

### Data preparation

Data preparation comprises two steps:
- a data defintion check, which checks if your date fits the given definiton, and
- time series creation, which creates the time series that will be used for the forecasting.

In [ ]:
import futureexpert.checkin as checkin

data_definition = DataDefinition(remove_rows=[],
                                 remove_columns=[],
                                 date_column=checkin.DateColumn(name='Date', format='%d.%m.%Y'),
                                 value_columns=[checkin.ValueColumn(name='Umsatz', name_new='Sales')],
                                 group_columns=[checkin.GroupColumn(name='land', name_new='Country')])

user_input_id = upload_feedback['uuid']
file_id = upload_feedback['files'][0]['uuid']

data_definition_feedback = client.check_data_definition(user_input_id=user_input_id,
                                                        file_uuid=file_id,
                                                        data_definition=data_definition,
                                                        file_specification=FileSpecification(delimiter=";"))
pp.pprint(data_definition_feedback)

In [ ]:
config_ts_creation = TsCreationConfig(time_granularity='monthly', value_columns_to_save=[
                                      'Sales'], missing_value_handler='setToZero')

create_ts_feedback = client.create_time_series(user_input_id=user_input_id,
                                               file_uuid=file_id,
                                               data_definition=data_definition,
                                               config_ts_creation=config_ts_creation,
                                               file_specification=FileSpecification(delimiter=";"))
pp.pprint(create_ts_feedback)

### Start the forecast

To start the forecast, you need to give
- the time series identification number, received from the time series creation task, and
- the report configuration.

In [ ]:
fc_report_config = ReportConfig(title='My first report', forecasting=ForecastingConfig(fc_horizon=5))

version = create_ts_feedback['result']['tsVersion']
forecast_identifier = client.start_forecast(version=version, config=fc_report_config)

### See the list of reports

In the meantime, you can list your reports, including the one you just requested.

In [ ]:
client.get_reports(limit=1)

## Inspect the forecasting results

In [ ]:
import time

while not (current_status := client.get_report_status(id=forecast_identifier)).is_finished:
    current_status.print()
    print('Waiting another 30 seconds to finish forecasting...')
    time.sleep(30)  # Wait between status requests

# Retrieve the final results
results_from_version_id = client.get_fc_results(
    id=forecast_identifier, include_backtesting=True, include_k_best_models=2)
